In [1]:
from ffpyplayer.player import MediaPlayer

In [2]:
from PyQt5.QtCore import QDir, Qt, QUrl
from PyQt5.QtMultimedia import QMediaContent, QMediaPlayer
from PyQt5.QtMultimediaWidgets import QVideoWidget
from PyQt5.QtWidgets import (QApplication, QFileDialog, QHBoxLayout, QLabel,
        QPushButton, QSizePolicy, QSlider, QStyle, QVBoxLayout, QWidget)
from PyQt5.QtWidgets import QMainWindow,QWidget, QPushButton, QAction
from PyQt5.QtGui import QIcon


In [3]:
from progress.spinner import Spinner
from PyQt5.QtCore import *
from PyQt5.uic import loadUi
from PyQt5.QtGui import QImage ,QPixmap ,QMovie 
from PyQt5.QtWidgets import QDialog, QApplication, QMainWindow, QMessageBox, QDialogButtonBox, QVBoxLayout, QFileDialog 


from PyQt5 import QtWidgets, QtMultimedia, uic, QtCore
from PyQt5 import QtCore, QtGui, QtWidgets

import subprocess
import time
import sys
import cv2
import datetime
from googletrans import Translator
import speech_recognition as sr
import threading

In [4]:
class CustomDialog(QDialog):

    def __init__(self, *args, **kwargs):
        super(CustomDialog, self).__init__(*args, **kwargs)
        
        self.setWindowTitle("HELLO!")
        
        QBtn = QDialogButtonBox.Ok | QDialogButtonBox.Cancel
        self.flag=False
        self.buttonBox = QDialogButtonBox(QBtn)
        self.buttonBox.accepted.connect(self.accept)
        self.buttonBox.rejected.connect(self.reject)
        self.pushButton.clicked.connect(self.checkstatus) 
        self.layout = QVBoxLayout()
        self.layout.addWidget(self.buttonBox)
        self.setLayout(self.layout)

        
class application(QMainWindow):
    
    datafile=""
    translatedfile=""
    flag=0
    datalist=[]
    language={ 'Hindi':"hi",
               'Bengali':"bn",
               'Gujarati':"gu",
               'Kannada':"kn",
               'Marathi':"mr",
               'Tamil':"ta",
               'Telugu':"te",
               }
    dic={'text':'*.txt | *.docx', 'audio':'*.mp3', 'video':'*.mp4 | *.wav'}
    font=14
    def __init__(self):
        super(application,self).__init__()
        loadUi('finalapp.ui',self)
        self.stack.setCurrentIndex(0)
        
        #   page1 buttons
        self.microphone.clicked.connect(self.recordclicked)
        self.camera.clicked.connect(lambda :self.stack.setCurrentIndex(1))
        
        self.p1.clicked.connect(lambda :self.stack.setCurrentIndex(0))
        self.p2.clicked.connect(self.translateclicked)
        self.p3.clicked.connect(self.save)
        
        self.text.clicked.connect(self.upload)
        self.audio.clicked.connect(self.upload)
        self.video.clicked.connect(self.upload)
        
        
        self.pushButton.clicked.connect(self.checkstatus)
        #   page2 buttons
        self.record.clicked.connect(self.recordclicked)
        
        
        #page4 buttons
        self.pushButton_2.clicked.connect(self.generatesubtitles)
        
        # hiding elements
        
        
   
    # while using microphone for live speech recognition
    def recordclicked(self):
        r = sr.Recognizer()
        m = sr.Microphone()
        r.pause_threshold=0.3
        r.non_speaking_duration=0.1
        self.datalist.clear()
        with m as source:
            r.adjust_for_ambient_noise(source)
        def callback(recognizer, audio):
                # received audio data, now we'll recognize it using Google Speech Recognition
            try:
                #r.pause_threshold = 1.0
                data=r.recognize_google(audio)
                print(data)
                #self.datalist.append(data)
                self.plainTextEdit_4.appendPlainText(data)
            except sr.UnknownValueError:
                    print("Google Speech Recognition could not understand audio")
            except sr.RequestError as e:
                    print("Could not request results from Google Speech Recognition service; {0}".format(e))
        
        stop_listening=r.listen_in_background(m, callback)
        print("recording started")
        
        if self.microphone.isChecked()==False:
            print("microphone stopping")
            stop_listening(wait_for_stop=False)#self.print_thread = threading.Thread(target=self.printing)
        #self.print_thread.start()
        #self.movie.setCacheMode(QMovie.CacheAll)
        #self.movie1.start()
        
        
        
    def printing(self):
        length=0
        while True:
            if len(self.datalist)>length:
                print(length," ",self.datalist[length])
                self.plainTextEdit_5.appendPlainText(self.datalist[length])
                length=length+1
        
       
    
            
                
    def translateclicked(self):
        
        if self.cbox.currentIndex()==-1:
            msg=QMessageBox()
            msg.setIcon(QMessageBox.Warning)
            msg.setStandardButtons(QMessageBox.Ok | QMessageBox.Cancel)
            msg.setText("Select a language First!")
            #msg.setInformativeText("This is additional information")
            msg.setWindowTitle("Warning")
            #msg.setDetailedText("The details are as follows:")
            msg.exec_()
        else:
            self.lan = str(self.cbox.currentText())
            
            ci= self.stack.currentIndex() 
            if ci==0:
                translator = Translator()
                self.datafile=self.plainTextEdit_4.toPlainText()
                self.translatedfile=translator.translate(self.datafile, dest=self.language[self.lan]).text
                self.plainTextEdit_3.setPlainText(self.translatedfile)
            elif ci==3:
                
                translator = Translator()
                self.datafile=self.plainTextEdit.toPlainText()
                self.translatedfile=translator.translate(self.datafile, dest=self.language[self.lan]).text
                self.plainTextEdit_2.setPlainText(self.translatedfile)
            elif ci==4:
                x=0
                translator = Translator()
                with open("subtitles.txt", encoding='UTF-8') as f:
                    for line in f:
                        x=x+1
                        if x==3:
                            self.translatedfile=translator.translate(line, dest=self.language[self.lan]).text
                            self.plainTextEdit_8.appendPlainText(self.translatedfile)
                            x=-2
                        else:
                            self.plainTextEdit_8.appendPlainText(line)
                f.close()
                
                    
    def upload(self):
        self.filetype=str(self.sender().objectName())
        self.filename, _ = QFileDialog.getOpenFileName(None, caption='Select a ' + self.filetype + ' file', filter=self.dic[self.filetype])
        if self.filetype=="video":
            self.stack.setCurrentIndex(4)
            self.videoplayer()
            
        else:
            text=self.filename.split('/')[-1]
            self.label.setText(text)
            self.fileprint()
        
        
    def videoplayer(self):
        cap = cv2.VideoCapture(self.filename) 
        player = MediaPlayer(self.filename)
        while(cap.isOpened()): 
            ret, frame = cap.read()
            if not ret:
                break
            audio_frame, val = player.get_frame()
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = QImage(frame, frame.shape[1], frame.shape[0], QImage.Format_RGB888)
            pix = QPixmap.fromImage(img)
            pix = pix.scaled(850, 650, Qt.KeepAspectRatio, Qt.SmoothTransformation)
            self.label_12.setPixmap(pix)    
            if val != 'eof' and audio_frame is not None:
            #audio
                img, t = audio_frame
            #self.ui.frame = pix  # or img depending what `ui.frame` needs
            if self.flag==True:
                break
                
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()
        
    def checkstatus(self):
        self.flag=not self.flag
        if self.flag==False:
            self.videoplayer()
            self.pushButton.setIcon(QIcon(r'C:\Users\GARG\Desktop\tanvi\project\majorproject\Resources\pause.jpg'))
        else:
            self.pushButton.setIcon(QIcon(r'C:\Users\GARG\Desktop\tanvi\project\majorproject\Resources\download.jpg'))
        
            
            
        

            
        
    def generatesubtitles(self):
        #converting .mp4 file into .wav format
        command1 = ["ffmpeg", "-y", "-i",self.filename, "-ac", "1", "-ar", "16000", r"C:\Users\GARG\Desktop\tanvi\project\majorproject\Samplefile\mono.wav"]
        subprocess.run(command1,shell=True,capture_output=True)

        #getting time stamps for each spoken word of generated audio file
        command2= r"C:\Users\GARG\Desktop\tanvi\project\majorproject\pocketsphinx\bin\Release\x64\pocketsphinx_continuous.exe -time yes -infile C:\Users\GARG\Desktop\tanvi\project\majorproject\Samplefile\mono.wav -hmm C:\Users\GARG\Desktop\tanvi\project\majorproject\pocketsphinx\model\en-us\en-us -lm C:\Users\GARG\Desktop\tanvi\project\majorproject\pocketsphinx\model\en-us\en-us.lm.bin -dict C:\Users\GARG\Desktop\tanvi\project\majorproject\pocketsphinx\model\en-us\cmudict-en-us.dict > C:\Users\GARG\Desktop\tanvi\project\majorproject\textDump.txt"
        subprocess.run(command2,shell=True,capture_output=True)
        
        print('complete1')
        #function to create subttiles
        self.createsubtitles()
        print('complete2')
        
        with open("subtitles.txt", encoding='UTF-8') as f:
                for line in f:
                    self.plainTextEdit_7.appendPlainText(line)
        f.close()
        
       
    def createsubtitles(self):
        f= open("C:/Users/GARG/Desktop/tanvi/project/majorproject/subtitles.txt","w+")

        #target_date_time_ms = 2000 # or whatever
        basetime = datetime.datetime( 1, 1, 1 )
        #delta = datetime.timedelta( 0, 0, 0, target_date_time_ms )
        #target_date = base_datetime + delta
        k=1
        file=r"C:\Users\GARG\Desktop\tanvi\project\majorproject\textDump.txt"
        lookup="<s>"
        lookdown="</s>"
        prevline=""
        data=""
        tt=""
        with open(file, 'r') as myFile:
            for num, line in enumerate(myFile, 1):
                if lookup in line:
                    data=prevline

                    sp=line.split()
                    start=str(basetime+datetime.timedelta( 0, 0, 0, float(sp[1])*1000))
                    #end=str(basetime+datetime.timedelta( 0, 0, 0, float(sp[2])*1000))
                    #print(start[0:12],end)
                    if start[20:23]=='':
                        sm="000"
                    else:
                        sm=start[20:23]
                   # if end[20:23]=='':
                        #m="000"
                    #else:
                        #m=end[20:23]
                    #print(start[0][11:19])
                    tt= start[11:19]+","+sm+" --> "#end[11:19],",",em," "

                    #print ('found at line:', num, line,prevline, sp[1],sp[2])
                    #=k+1
            
                if lookdown in line:
                    f.write("%d\n"%k)
                    sp=line.split()
                    start=str(basetime+datetime.timedelta( 0, 0, 0, float(sp[1])*1000))
                    if start[20:23]=='':
                        sm="000"
                    else:
                        sm=start[20:23]
                    print(tt,start[11:19],",",sm,sep="",file=f)#end[11:19],",",em," "
                    f.write("%s\n\n"%data)
                    tt=""
                    k=k+1
          
            
            
            
                prevline=line
        f.close()    
        
        
   
        
    
    
    def fileprint(self):
        if self.filetype=="text":
            self.textfileprint_thread = threading.Thread(target=self.textfileprint)
            self.textfileprint_thread.start()
            #time.sleep(0.25)
                    
        if self.filetype=="audio":
            msg=QMessageBox()
            msg.setIcon(QMessageBox.Warning)
            msg.setStandardButtons(QMessageBox.Ok)
            msg.setText("Please wait your file is processing....")
            #msg.setInformativeText("This is additional information")
            msg.setWindowTitle("Warning")
            #msg.setDetailedText("The details are as follows:")
            msg.exec_()
            #if self.flag==1:
                
            
            
            self.speechrecognitionprint_thread = threading.Thread(target=self.speechrecognitionprint)
            self.speechrecognitionprint_thread.start()
            #self.movie1=QMovie("resources/spinner1.gif", QByteArray(), self)
            #self.label_10.setMovie(self.movie1)
            #self.movie1.start()
            #self.label_10.show()
            #time.sleep(0.25)
            

    
    def save(self):
        
        file, _ = QFileDialog.getSaveFileName(self,"Save File",filter="*.txt")
        #name = QtGui.QFileDialog.getSaveFileName(self, 'Save File')
        f = open(file,'w+',encoding='utf-8')
        
        f.write(self.translatedfile)
        f.close()
        msg=QMessageBox()
        msg.setIcon(QMessageBox.Information)
        msg.setStandardButtons(QMessageBox.Ok)
        msg.setText("File saved successfully!")
        #msg.setInformativeText("This is additional information")
        msg.setWindowTitle("Warning")
            #msg.setDetailedText("The details are as follows:")
        msg.exec_()
        
########################################        THREADS             #######################################################
    
    def microphoneinput(self):
        m = sr.Microphone()
        r = sr.Recognizer()
        while 1:
            with m as source:
                #r.adjust_for_ambient_noise(source)
                audio = r.listen(source)
       
            
            data=r.recognize_google(audio)
        
            self.plainTextEdit_4.setPlainText(data)
        self.flag=1
        self.movie.stop()
        
        
        
    def textfileprint(self):
        with open(self.filename, encoding='UTF-8') as f:
                for line in f:
                    if line.strip():
                        self.plainTextEdit.appendPlainText(line)
        
        
    def speechrecognitionprint(self):
        command = ["ffmpeg", "-y", "-i",self.filename, "-ac", "1", "-ar", "16000", r"C:\Users\GARG\Desktop\tanvi\project\majorproject\Samplefile\mono.wav"]
        subprocess.run(command,shell=True,capture_output=True)
        r = sr.Recognizer()
        with sr.AudioFile(r"C:\Users\GARG\Desktop\tanvi\project\majorproject\Samplefile\mono.wav") as source:
            r.adjust_for_ambient_noise(source)
            audio = r.record(source)
            try:
                self.datafile=r.recognize_sphinx(audio)
                #print(type(datafile))
                #print("Sphinx thinks you said " + datafile)
            except sr.UnknownValueError:
                print("Sphinx could not understand audio")
            except sr.RequestError as e:
                print("Sphinx error; {0}".format(e))
        
        
        print("completed")
        self.plainTextEdit.appendPlainText(self.datafile)
        self.stack.setCurrentIndex(3)
        
        

In [5]:
app=QApplication(sys.argv)
window=application()
window.show()
try:
    sys.exit(app.exec_())
except:
    print('exiting')

complete1
complete2
complete1
complete2
exiting
